Hey, hope you're doing well. This project will analyze EV charging utilization metrics from a public Kaggle dataset which measures EV charging station usage from Palo Alto.

To begin, let's first upload the necessary libraries:

In [2]:
import zipfile

from pathlib import Path

Let's now unzip this dataset locally here (it is a zip file as it is an extremely large dataset).

In [4]:
zip_path = Path("data/EVChargingStationUsage.csv.zip")

extract_dir = Path("data/extracted") # Placing the unzipped CSV in a temporary folder

# Let's now unzip this file:

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_dir)

list(extract_dir.glob("*"))

# Let's check if that extraction worked.    

[PosixPath('data/extracted/EVChargingStationUsage.csv')]

Because this dataset tracks usages, it is an event-based and relational system. In practice, each usage belongs to exactly one station, and each station relates to multiple usages. We can create relational tables for this, and we will use SQL to do so.

In [5]:
import sqlite3

db_path = Path("data/ev_charging.db") # to create a file path object
conn = sqlite3.connect(db_path) # now let's connect to the database
print("DB created at:", db_path.resolve())

conn.execute("SELECT 1").fetchone() # as a test

DB created at: /home/jupyter/ev-charging-station-analysis/ev-charging-stations-analysis/data/ev_charging.db


(1,)